In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#%matplotlib inline
from bs4 import BeautifulSoup
import html5lib
import requests
import datetime
import time
from f1 import Team
from f1 import Race
import matplotlib.animation as animation

In [3]:
#Change these variables to update the race:
RACE = '7'
RACENAME = 'Canada'

In [4]:
#This section is to download fresh results from Ergast
path = 'http://ergast.com/api/f1'
season = '2016'
URL = path + '/' + season + '/' + RACE + '/' + 'laps?limit=1000'

#Download
results = requests.get(URL)
#soup = BeautifulSoup(results.content, 'lxml')
print("First 1000 records downloaded")
#Part 2
URL = path + '/' + season + '/' + RACE + '/' + 'laps?limit=1000&offset=1000'
results2 = requests.get(URL)
#soup2 = BeautifulSoup(results2.content, 'lxml')
with open(RACENAME + '_race_dl.xml', 'wb') as f:
    f.write(results.content)
    f.write(results2.content)

First 1000 records downloaded


In [5]:
#This section will just open a saved copy.  Run it always to pull the composite soup
with open(RACENAME + '_race_dl.xml', 'rb') as f:
    results = f.read()
soup = BeautifulSoup(results, 'lxml')

In [80]:
#pull in driverids to names
driver_info = pd.DataFrame.from_csv('driver_info.csv')
driverid_dict = {driver_info.ix[i]['driverid']:driver_info.ix[i]['GivenName'] + " " + driver_info.ix[i]['FamilyName'] for i in driver_info.index}
driverid_dict['sainz'] = driverid_dict['sainz'] + ' Jr.'
colors = {driver_info.ix[i]['GivenName'] + " " + driver_info.ix[i]['FamilyName']: driver_info.ix[i]['color'] for i in driver_info.index}
#Manually Correct Carlos Sainz
colors['Carlos Sainz Jr.'] = colors['Carlos Sainz']
driver_abrev = {driver_info.ix[i]['GivenName'] + " " + driver_info.ix[i]['FamilyName']:driver_info.ix[i]['code'] for i in driver_info.index}
driver_abrev['Carlos Sainz Jr.'] = 'SAI'

In [81]:
headers = ('driverid', 'lap', 'time')
num_rows = len(soup.find_all('timing'))
race_dict = {}
for header in headers:
    race_dict[header] = [race[header] for race in soup.find_all('timing')]
df = pd.DataFrame(race_dict)
df['minutes'] = df['time'].apply(lambda x: x.split(':')[0])
df['seconds'] = df['time'].apply(lambda x: x.split(':')[1])
df['lap_sec'] = df['minutes'].astype('int64') * 60 + df['seconds'].astype('float32')
df['lap'] = df['lap'].astype('int64')
df['driver'] = df['driverid'].apply(lambda x: driverid_dict[x])

In [82]:
num_rows

1371

In [83]:
pivot = df.pivot(index='driver', columns='lap', values='lap_sec')

In [84]:
for i in range(2, df['lap'].max() + 1):
    pivot.ix[:,i] = pivot.ix[:,i] + pivot.ix[:, i-1]

In [85]:
unpivot = pivot.stack()
new_df = unpivot.reset_index()
new_df.columns = ['driverid', 'lap', 'cume_sec']
new_df.sort_values('cume_sec', ascending=True, inplace=True)
new_df = new_df.reset_index()
new_df.drop('index', axis=1, inplace=True)

In [86]:
#Set up a Race object, score it, return a dataframe of all points
ThisRace = Race(RACENAME)
df_score = ThisRace.score()
ThisTeam = Team(RACENAME)

In [87]:
#Set up current dataframe:
#tracks each drivers position at a certain point of the race
#and the laps completed.
current = pd.DataFrame(columns=['position', 'lap'])
current['position'] = df_score['Grid'].copy()
current['lap'] = 0
if 'DNS' in current['position']:
    current['position'][current['position'] == 'DNS'] = np.NaN
if 'PL' in current['position']:
    current['position'][current['position'] == 'PL'] = np.NaN
#current['position']['Max Verstappen'] = 21
#current['position']['Felipe Nasr'] = 22
#current.sort_values('position', inplace=True)

In [92]:
# if 'PL' in current['position']:
#     print('True')
#current['position']['Max Verstappen']
#current['position']['Felipe Nasr'] = np.NaN
#current['position'] = np.arange(22) + 1

# current['position']['Kevin Magnussen'] = 23
#current.sort_values(columns='position', inplace=True)
# current['position'] = np.arange(22) + 1


C:\Users\Jason\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,position,lap
Lewis Hamilton,1,0
Nico Rosberg,2,0
Sebastian Vettel,3,0
Daniel Ricciardo,4,0
Max Verstappen,5,0
Kimi Räikkönen,6,0
Valtteri Bottas,7,0
Felipe Massa,8,0
Nico Hülkenberg,9,0
Fernando Alonso,10,0


In [93]:
#Set all laps completed to zero
ThisRace.laps_completed = {driver:current.ix[driver]['lap'] for driver in current.index}

In [94]:
new_df

,driverid,lap,cume_sec
0,Sebastian Vettel,1,86.080999
1,Lewis Hamilton,1,86.378000
2,Max Verstappen,1,87.405001
3,Daniel Ricciardo,1,88.643000
4,Kimi Räikkönen,1,89.358999
5,Valtteri Bottas,1,89.771999
6,Felipe Massa,1,90.372999
7,Fernando Alonso,1,90.857000
8,Nico Hülkenberg,1,91.271000
9,Nico Rosberg,1,91.889000


In [95]:
#Reworking on real time
fastest_lap_time = 1000 #dummy value
first = False 
row_count = 0
fps = 5

def blank():
    pass
def update_driver_scores2(step, rects, ax):
    global fastest_lap_time
    global ThisRace
    global current
    global new_df
    global first
    global row_count
    global fps
    #print('Step:', step, 'Row Count:', row_count)
    if float(new_df.ix[row_count]['cume_sec']) < step / fps:
        driver, lap, cume_sec = new_df.ix[row_count]
        if lap == 1:
            if cume_sec != np.NaN:
                cur_lap_time = cume_sec
        elif new_df.ix[row_count, 'cume_sec'] != np.NaN:
            cur_lap_time = cume_sec - float(new_df['cume_sec'][(new_df['driverid'] == driver) & (new_df['lap'] == lap - 1)])
        if cur_lap_time < fastest_lap_time: #check for fastest lap
            ThisRace.fastest_lap = driver
            fastest_lap_time = cur_lap_time
        current.ix[driver, 'lap'] += 1 #increment lap
        current.sort_values('lap', kind='mergesort', inplace=True, ascending=False) #sort by lap, stable sort
        current['position'] = np.arange(1,len(current) + 1) #recalc positions
        ThisRace.laps_completed[driver] = lap #update laps completed for driver
        ThisRace.drivers_position = {driver:current.ix[driver, 'position'] for driver in current.index} #update driver positions
        ThisRace.score().sort_values('Finish', inplace=True, ascending=True)   
        for rect, h, driver in zip(rects, 
                                   [ThisRace.fantasy_points[driver] for driver in current.index], 
                                   [driver for driver in current.index]):
            rect.set_height(h)
            rect.set_color(colors[driver])
            rect.set_edgecolor('#000000')
        ax.xaxis.set_ticklabels([driver_abrev[driver] for driver in current.index])
        #ax.xaxis.set_ticklabels(current.index)
        row_count += 1

#whole race
start_time = datetime.datetime.now()
end_time = start_time + datetime.timedelta(seconds=float(new_df[-1:]['cume_sec']))
ThisRace.drivers_position = ThisRace.grid_position
ThisRace.score()


fig1 = plt.figure()
#fig, ax = plt.subplots()
ax = fig1.add_subplot(1,1,1)
#ax.grid(False)
ax.xaxis.grid(False)
ax.set_frame_on(False)
ax.tick_params(axis='both', width=0)
x_pos = current['position']
points = [ThisRace.fantasy_points[driver] for driver in current.index]
rects = ax.bar(x_pos, points, align='center', edgecolor='#000000', linewidth=0.5)
plt.xticks(x_pos, [driver_abrev[driver] for driver in current.index], rotation=90)
plt.xlim([0, len(points)+1])
plt.title('Drivers Fantasy Points')
plt.ylabel('Points')
plt.ylim([0, 25])
interval = int(new_df[-1:]['cume_sec'] * fps)
# ax.spines['top'].set_visible(True)
# ax.spines['top'].set_linewidth(3)
for rect, h, driver in zip(rects, 
                                   [ThisRace.fantasy_points[driver] for driver in current.index], 
                                   [driver for driver in current.index]):
            rect.set_height(h)
            rect.set_color(colors[driver])
            rect.set_edgecolor('#000000')
ani = animation.FuncAnimation(fig1, update_driver_scores2, interval, fargs=([rects, ax]), init_func=blank)
ani.save(RACENAME + '2_movie.mp4', fps=fps, writer='ffmpeg', bitrate=5000)
#ani.save('China_movie.mp4', fps=5, writer='ffmpeg')
#ani.save('China_movie.mp4', fps='1', writer='ffmpeg', savefig_kwargs={'facecolor':'red'})
#plt.show()
#ani.save('china_test.mp4', writer='ffmpeg', fps=1)

In [ ]:
interval

In [ ]:
len(new_df)

In [ ]:
#ani.save('China_realtime_movie.mp4', fps=fps, writer='ffmpeg')

In [ ]:
# fastest_lap_time = 1000 #dummy value
# first = False 

# def update_driver_scores2(step, rects, ax):
#     global fastest_lap_time
#     global ThisRace
#     global current
#     global new_df
#     global first
#     #init function is called twice initially.  First call gets voided
#     if first == False:
#         first = True
#     else:
#         driver, lap, cume_sec = new_df.ix[step]
#         if lap == 1:
#             if cume_sec != np.NaN:
#                 cur_lap_time = cume_sec
#         elif new_df.ix[i, 'cume_sec'] != np.NaN:
#             cur_lap_time = cume_sec - float(new_df['cume_sec'][(new_df['driverid'] == driver) & (new_df['lap'] == lap - 1)])

#         if cur_lap_time < fastest_lap_time: #check for fastest lap
#             ThisRace.fastest_lap = driver
#             fastest_lap_time = cur_lap_time
#         current.ix[driver, 'lap'] += 1 #increment lap
#         current.sort_values('lap', kind='mergesort', inplace=True, ascending=False) #sort by lap, stable sort
#         current['position'] = np.arange(1,len(current) + 1) #recalc positions
#         ThisRace.laps_completed[driver] = lap #update laps completed for driver
#         ThisRace.drivers_position = {driver:current.ix[driver, 'position'] for driver in current.index} #update driver positions
#         ThisRace.score().sort_values('Finish', inplace=True, ascending=True)   
#         for rect, h, driver in zip(rects, 
#                                    [ThisRace.fantasy_points[driver] for driver in current.index], 
#                                    [driver for driver in current.index]):
#             rect.set_height(h)
#             rect.set_color(colors[driver])
#             rect.set_edgecolor('#000000')
#         ax.xaxis.set_ticklabels([driver_abrev[driver] for driver in current.index])
#         #ax.xaxis.set_ticklabels(current.index)
# start_time = datetime.datetime.now()

# #whole race
# end_time = start_time + datetime.timedelta(seconds=float(new_df[-1:]['cume_sec']))
# ThisRace.drivers_position = ThisRace.grid_position
# ThisRace.score()

# def blank():
#     pass
# fig1 = plt.figure()
# #fig, ax = plt.subplots()
# ax = fig1.add_subplot(1,1,1, axisbg='red')
# ax.grid(False)
# ax.set_frame_on(False)
# ax.tick_params(axis='both', width=0)
# ax.set_axis_bgcolor('r')
# x_pos = current['position']
# points = [ThisRace.fantasy_points[driver] for driver in current.index]
# rects = ax.bar(x_pos, points, align='center', edgecolor='#000000', linewidth=0.5)
# plt.xticks(x_pos, current.index, rotation=90)
# plt.xlim([0, len(points)+1])
# plt.title('Drivers Fantasy Points')
# plt.ylabel('Points')
# plt.ylim([0, 25])
# # ax.spines['top'].set_visible(True)
# # ax.spines['top'].set_linewidth(3)
# ani = animation.FuncAnimation(fig1, update_driver_scores2, 1000, fargs=([rects, ax]))
# ani.save('China_movie.mp4', fps='5', writer='ffmpeg')
# #ani.save('China_movie.mp4', fps='1', writer='ffmpeg', savefig_kwargs={'facecolor':'red'})
# #plt.show()
# #ani.save('china_test.mp4', writer='ffmpeg', fps=1)

In [ ]:
# figtest = plt.figure()
# ax = figtest.add_subplot(1,1,1)
# x_pos = current['position']
# points = [ThisRace.fantasy_points[driver] for driver in current.index]
# ax.bar(x_pos, points, align='center', edgecolor='#000000', linewidth=)
# for rect in rects:
#     rect.linewidth = 3
#     rect.edgecolor = '#000000'
#plt.show()
#animation.FuncAnimation()
#ani.save('china_test.mpeg', writer='ffmpeg')

In [ ]:
# fig2, axarr = plt.subplots(2,1)
# axarr[0,0].set_title('Driver Points')
# axarr[0,0].plot(current.index, current['position'])
# axarr[0,1].set_title('Team Points')
# axarr[0,1].plot()
# axarr[1,0].set_title('Team Points Over Time')

# plt.show()

In [ ]:
#ThisTeam = Team('China')

In [ ]:
#team_df = pd.DataFrame(columns=['Team', 'Points'])
#team_df['Team'] = ThisTeam.teams.keys()
#team_df['Points'] = [ThisRace.fantasy_points[driver] for driver in ThisTeam.teams.]

In [ ]:
# import matplotlib.animation as animation

# def update_line(num, data, line):
#     line.set_data(data[...,:num])
#     return line,

# fig1 = plt.figure()

# data = np.random.rand(2, 25)
# l, = plt.plot([], [], 'r-')
# plt.xlim(0, 1)
# plt.ylim(0, 1)
# plt.xlabel('x')
# plt.title('test')
# line_ani = animation.FuncAnimation(fig1, update_line, 25, fargs=(data, l),
#     interval=50, blit=True)
# #line_ani.save('lines.mp4')

# fig2 = plt.figure()

# x = np.arange(-9, 10)
# y = np.arange(-9, 10).reshape(-1, 1)
# base = np.hypot(x, y)
# ims = []
# for add in np.arange(15):
#     ims.append((plt.pcolor(x, y, base + add, norm=plt.Normalize(0, 30)),))

# im_ani = animation.ArtistAnimation(fig2, ims, interval=50, repeat_delay=3000,
#     blit=True)
# #im_ani.save('im.mp4', metadata={'artist':'Guido'})

# plt.show()

In [ ]:
# import matplotlib.animation as animation
# def update_driver_scores(raceObj, current, driver, lap, lap_time, abs_time):
#     #do I need globals so the objects are edited?
#     current.ix[driver, 'lap'] += 1 #increment lap
#     current.sort_values('lap', kind='mergesort', inplace=True, ascending=False) #sort by lap, stable
#     current['position'] = np.arange(1,len(current) + 1) #recalc positions
#     raceObj.laps_completed[driver] = lap #update laps completed for driver
#     raceObj.drivers_position = {driver:current.ix[driver, 'position'] for driver in current.index} #update driver positions
#     if lap_time < fastest_lap_time: #check for fastest lap
#         raceObj.fastest_lap = driver
#         fastest_lap_time = cur_lap_time
#     return raceObj.score().sort_values('Finish', inplace=True, ascending=True)

# def update_bar_chart(racePanda, rects):
#     for rect, h in zip(rects, racePanda['Total_Race_Pts']):
#         rect.set_height(h)

# fastest_lap_time = 1000
# start_time = datetime.datetime.now()
# #whole race
# end_time = start_time + datetime.timedelta(seconds=float(new_df[-1:]['cume_sec']))
# i = 0
# ThisRace.drivers_position = ThisRace.grid_position
# ThisRace.score()

# fig1 = plt.figure()
# ax = fig1.add_subplot(1,1,1)
# x_pos = current['position']
# points = [ThisRace.fantasy_points[driver] for driver in current.index]
# rects = plt.bar(x_pos, points, align='center', alpha=0.7)
# plt.xticks(x_pos, current.index, rotation=90)
# plt.xlim([-1, len(points)])
# plt.title('Drivers Fantasy Points')
# plt.xlabel('Driver')
# plt.ylabel('Points')
# plt.ylim([0, 25])
# #ani = animation.FuncAnimation(fig1, update_driver_scores, i, fargs=(ThisRace, ))
# plt.show()